# Topos 
## Data Engineering Internship Assessment
## Ravi Patel   |   Date: 05/22/2019
------

## 0. Importing all libraries
----

In [1]:
import sys
import requests
import re
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver

## 1. Creating a Client
-----
Since Wikipages and Google pages contain empty tags that will hold dynamic content. Requesting the page regularly using requests library will only return html without running javascript resulting in empty tags.

In [2]:
WIKI_TOP_CITIES = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
WIKIURL = 'https://en.wikipedia.org'

## 2. Requesting The Client

In [3]:
req = requests.get(WIKI_TOP_CITIES)
soup = BeautifulSoup(req.content, 'lxml')

### 2.1 Create driver and open WIKI TOP CITIES

In [4]:
driver = webdriver.Firefox()
driver.get(WIKI_TOP_CITIES)
html = driver.execute_script("return document.documentElement.outerHTML")
sel_soup = BeautifulSoup(html, 'lxml')
print(sel_soup.find('body').prettify())


<body class="mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject page-List_of_United_States_cities_by_population rootpage-List_of_United_States_cities_by_population skin-vector action-view">
 <div class="noprint" id="mw-page-base">
 </div>
 <div class="noprint" id="mw-head-base">
 </div>
 <div class="mw-body" id="content" role="main">
  <a id="top">
  </a>
  <div class="mw-body-content" id="siteNotice">
   <!-- CentralNotice -->
  </div>
  <div class="mw-indicators mw-body-content">
   <div class="mw-indicator" id="mw-indicator-featured-star">
    <a href="/wiki/Wikipedia:Featured_lists" title="This is a featured list. Click here for more information.">
     <img alt="This is a featured list. Click here for more information." data-file-height="438" data-file-width="462" decoding="async" height="19" src="//upload.wikimedia.org/wikipedia/en/thumb/e/e7/Cscr-featured.svg/20px-Cscr-featured.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/e/e7/Cscr-featured.svg/30px-Cscr-f

### 2.2 Save the table with important information

In [5]:
table = sel_soup.findAll('table',{'class':'wikitable'})
print(table[1].prettify())

<table class="wikitable sortable" style="text-align:center">
 <tbody>
  <tr>
   <th>
    2018
    <br/>
    rank
   </th>
   <th>
    City
   </th>
   <th>
    State
    <sup class="reference" id="cite_ref-5">
     <a href="#cite_note-5">
      [c]
     </a>
    </sup>
   </th>
   <th>
    2018
    <br/>
    estimate
   </th>
   <th>
    2010
    <br/>
    Census
   </th>
   <th>
    Change
   </th>
   <th colspan="2">
    2016 land area
   </th>
   <th colspan="2">
    2016 population density
   </th>
   <th>
    Location
   </th>
  </tr>
  <tr>
   <td>
    1
   </td>
   <td style="text-align:left;background-color:#cfecec">
    <i>
     <a href="/wiki/New_York_City" title="New York City">
      New York City
     </a>
    </i>
    <sup class="reference" id="cite_ref-6">
     <a href="#cite_note-6">
      [d]
     </a>
    </sup>
   </td>
   <td align="left">
    <span class="flagicon">
     <img alt="" class="thumbborder" data-file-height="450" data-file-width="900" decoding="async" h

### 2.3 In the table, get all the table row information

In [6]:
tr = table[1].findAll('tr')
elements = tr[1:]

### 2.4 Process each table row to extract information
##### Name of City, State, Wiki link, etc. (go to wiki page)

In [7]:
city_id = []
city_name = []
city_state = []
city_wiki_link = []
city_latitude = []
city_longitude = []
city_size = []
city_population = []

In [9]:
city_population = []
for i,element in enumerate(elements):
    element = str(element).replace('\n','').replace('>','>\n').replace('<','\n<').replace('<b>','\n').replace('<b/>','\n').replace('<i>','\n').replace('</i>','\n').replace('[d]','')
    element_list = element.split('\n')
    
    element_list= [e for e in element_list if e != '']
    
    city_id.append(element_list[2])
    #--------------------------------------------------------------
    city_name.append(element_list[6])
    
    if re.search('\d',element_list[19]):
        city_state.append(element_list[15])
    elif re.search('<td .*>', element_list[19]):
        if re.search('<td .*>',element_list[16]):
            if re.search('</td>',element_list[14]):
                city_state.append(element_list[20])
            else:
                city_state.append(element_list[14])
        else:
            city_state.append(element_list[16])
    elif re.search('<a .*>', element_list[19]):
        city_state.append(element_list[20])
    elif re.search('</td>', element_list[19]):
        city_state.append(element_list[15])
    elif re.search('</span>', element_list[19]):
        if re.search('</td>', element_list[21]):
            city_state.append(element_list[20])
        else:
            city_state.append(element_list[21])
    else:
        city_state.append(element_list[19])
    
    
    city_wiki_link.append(WIKIURL+'/wiki/'+str(element_list[6]).replace(" ","_"))
    #------------------------------------------------------------------
    
    if re.search('.*"N', element_list[60]):
        city_latitude.append(element_list[60])
    elif re.search('.*"S', element_list[60]):
        city_latitude.append(element_list[60])
    elif re.search('.*W', element_list[60]):
        city_latitude.append(element_list[56])
    elif re.search('.*E', element_list[60]):
        city_latitude.append(element_list[56])
    elif re.search('<span class="latitude">', element_list[60]):
        city_latitude.append(element_list[61])
    elif re.search('<span class="longitude">', element_list[60]):
        city_latitude.append(element_list[57])
    elif re.search('</span>', element_list[60]):
        if re.search('<span class="latitude">', element_list[54]):
            city_latitude.append(element_list[55])
        elif re.search('<span class="plainlinks nourlexpansion">', element_list[54]):
            city_latitude.append(element_list[59])
        else:
            city_latitude.append(element_list[54])
    elif re.search('<span class="geo-dms" title="Maps, aerial photos, and other data for this location">', element_list[60]):
        city_latitude.append(element_list[62])
    else:
        city_latitude.append(element_list[60])
    
        
    #--------------------------------------------------
    
    if re.search('.*W', element_list[64]):
        city_longitude.append(element_list[64])
    elif re.search('.*E', element_list[64]):
        city_longitude.append(element_list[64])
    elif re.search('<span class="geo-multi-punct">', element_list[64]):
        city_longitude.append(element_list[60])
    elif re.search('<span class="longitude">', element_list[64]):
        city_longitude.append(element_list[65])
    elif re.search('</span>', element_list[64]):
        if re.search('</span>', element_list[58]):
            city_longitude.append(element_list[61])
        elif re.search('<span class="latitude">', element_list[58]):
            city_longitude.append(element_list[63])
        else:
            city_longitude.append(element_list[58])
    elif re.search('/', element_list[64]):
        city_longitude.append(element_list[59])
    elif re.search('', element_list[64]):
        city_longitude.append(element_list[66])
    else:
        city_longitude.append(element_list[64])
    
    
    #---------------------------------------------------------------
    
    if re.search('<sup>', element_list[36]):
        city_size.append(element_list[32])
    elif re.search('<td .*>', element_list[36]):
        city_size.append(element_list[37])
    elif re.search('km', element_list[36]):
        city_size.append(element_list[33])
    elif re.search('</sup>', element_list[36]):
        city_size.append(element_list[30])
    elif re.search('</td>', element_list[36]):
        if re.search('</span>', element_list[35]):
            city_size.append(element_list[38])
        else:
            city_size.append(element_list[35])
    elif re.search('2', element_list[36]):
        city_size.append(element_list[31])
    else:
        city_size.append(element_list[36])
    
    #-------------------------------------------------
    
    if re.search('</td>', element_list[23]):
        if re.search('California', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Kentucky', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Missouri', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Arizona', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Virginia', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Georgia', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('North Carolina', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Kansas', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('New Jersey', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Michigan', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Louisiana', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('Florida', element_list[19]):
            city_population.append(element_list[22])
        elif re.search('</span>', element_list[19]):
            city_population.append(element_list[25])
        else:
            city_population.append(element_list[19])
    elif re.search('<td .*>', element_list[23]):
        if re.search('<span .* style="display:none">', element_list[24]):
            city_population.append(element_list[17])
        else:
            city_population.append(element_list[24])
    elif re.search('<span data-sort-value=".*" style="display:none">', element_list[23]):
        city_population.append(element_list[17])
    else:
        city_population.append(element_list[23])
    
 

## 3. Top Ten Cities

In [10]:
top_city = city_id[0:10]
top_city_name = city_name[0:10]
top_city_state = city_state[0:10]
top_city_wiki_link = city_wiki_link[0:10]
top_city_latitude = city_latitude[0:10]
top_city_longitude = city_longitude[0:10]
top_city_size = city_size[0:10]
top_city_population = city_population[0:10]

In [19]:
import pandas as pd
top_city_table = pd.DataFrame()

In [20]:
top_city_table['City ID'] = top_city
top_city_table['City Name'] = top_city_name
top_city_table['State'] = top_city_state
top_city_table['Wiki Link'] = top_city_wiki_link
top_city_table['Latitude'] = top_city_latitude
top_city_table['Longitude'] = top_city_latitude
top_city_table['City Size'] = top_city_size
top_city_table['City Population'] = top_city_population

In [235]:
top_city_table['Wiki Link'][4]="https://en.wikipedia.org/wiki/Phoenix,_Arizona"
top_city_table['Wiki Link'][9] = 'https://en.wikipedia.org/wiki/San_Jose,_California'

In [236]:
top_city_table

,City Name,State,Wiki Link,Latitude,Longitude,City Size,City Population
0,New York City,New York,https://en.wikipedia.org/wiki/New_York_City,40°39′49″N,40°39′49″N,301.5 sq mi,"8,398,748"
1,Los Angeles,California,https://en.wikipedia.org/wiki/Los_Angeles,34°01′10″N,34°01′10″N,468.7 sq mi,"3,990,456"
2,Chicago,Illinois,https://en.wikipedia.org/wiki/Chicago,41°50′15″N,41°50′15″N,227.3 sq mi,"2,705,994"
3,Houston,Texas,https://en.wikipedia.org/wiki/Houston,29°47′12″N,29°47′12″N,637.5 sq mi,"2,325,502"
4,Phoenix,Arizona,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",33°34′20″N,33°34′20″N,517.6 sq mi,"1,445,632"
5,Philadelphia,Pennsylvania,https://en.wikipedia.org/wiki/Philadelphia,40°00′34″N,40°00′34″N,134.2 sq mi,"1,584,138"
6,San Antonio,Texas,https://en.wikipedia.org/wiki/San_Antonio,29°28′21″N,29°28′21″N,461.0 sq mi,"1,532,233"
7,San Diego,California,https://en.wikipedia.org/wiki/San_Diego,32°48′55″N,32°48′55″N,325.2 sq mi,"1,425,976"
8,Dallas,Texas,https://en.wikipedia.org/wiki/Dallas,32°47′36″N,32°47′36″N,340.9 sq mi,"1,345,047"
9,San Jose,California,"https://en.wikipedia.org/wiki/San_Jose,_Califo...",37°17′48″N,37°17′48″N,177.5 sq mi,"1,030,119"


## 4. Other Wiki Pages

In [11]:
def getPageHTML(url = ''):
    driver = webdriver.Firefox()
    driver.get(url)
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = BeautifulSoup(html, 'lxml')
    
    return sel_soup

### 4.1 City 1 : New York City

In [12]:
url1 = top_city_table['Wiki Link'][0]

In [29]:
city1_html = getPageHTML(url1)

In [120]:
city1_summary = city1_html.find_all('p')[1:38]
city1_geo = city1_html.find_all('p')[39:42]
city1_climate = city1_html.find_all('p')[51:54]

### 4.2 City 2 : LA

In [121]:
url2 = top_city_table['Wiki Link'][1]

In [122]:
city2_html = getPageHTML(url2)

In [137]:
city2_summary = city2_html.find_all('p')[2:24]
city2_geo = city2_html.find_all('p')[25:30]
city2_climate = city2_html.find_all('p')[42:51]

### 4.3 City 3 :Chicago

In [138]:
url3 = top_city_table['Wiki Link'][2]

In [139]:
city3_html = getPageHTML(url3)

In [148]:
city3_summary = city3_html.find_all('p')[2:39]
city3_geo = city3_html.find_all('p')[39:56]
city3_climate = city3_html.find_all('p')[56:58]

### 4.4 City 4 : Houston

In [149]:
url4 = top_city_table['Wiki Link'][3]

In [150]:
city4_html = getPageHTML(url4)

In [155]:
city4_summary = city4_html.find_all('p')[2:23]
city4_geo = city4_html.find_all('p')[24:27]
city4_climate = city4_html.find_all('p')[28:33]

### 4.5 City 5 : Phoenix

In [166]:
url5 = top_city_table['Wiki Link'][4]

In [167]:
city5_html = getPageHTML(url5)

In [199]:
city5_summary = city5_html.find_all('p')[1:29]
city5_geo = city5_html.find_all('p')[29:38]
city5_climate = city5_html.find_all('p')[38:43]

### 4.6 City 6 : Philadelphia

In [175]:
url6 = top_city_table['Wiki Link'][5]

In [176]:
city6_html = getPageHTML(url6)

In [198]:
city6_summary = city6_html.find_all('p')[1:24]
city6_geo = city6_html.find_all('p')[24:41]
city6_climate = city6_html.find_all('p')[41:47]

### 4.7 City 7 : San Antonio 

In [184]:
url7 = top_city_table['Wiki Link'][6]

In [185]:
city7_html = getPageHTML(url7)

In [206]:
city7_summary = city7_html.find_all('p')[1:22]
city7_geo = city7_html.find_all('p')[22:32]
city7_climate = city7_html.find_all('p')[32:37]

### 4.8 City 8 : San Diego

In [193]:
url8 = top_city_table['Wiki Link'][7]

In [194]:
city8_html = getPageHTML(url8)

In [207]:
city8_summary =  city8_html.find_all('p')[1:22]
city8_geo = city8_html.find_all('p')[22:31]
city8_climate = city8_html.find_all('p')[31:41]

### 4.9 City 9 : Dallas

In [208]:
url9 = top_city_table['Wiki Link'][8]

In [209]:
city9_html = getPageHTML(url9)

In [228]:
city9_summary = city9_html.find_all('p')[1:14]
city9_geo = city9_html.find_all('p')[14:29]
city9_climate = city9_html.find_all('p')[29:37]

### 4.10 City 10 : San Jose

In [238]:
url10 = top_city_table['Wiki Link'][9]

In [239]:
city10_html = getPageHTML(url10)

In [247]:
city10_summary = city10_html.find_all('p')[1:23]
city10_geo = city10_html.find_all('p')[23:34]
city10_climate = city10_html.find_all('p')[34:40]

## 5. Adding to the Table

In [248]:
summary = [city1_summary, city2_summary, city3_summary, 
           city4_summary, city5_summary, city6_summary,
           city7_summary, city8_summary, city9_summary, city10_summary]

In [249]:
geo = [city1_geo, city2_geo, city3_geo, 
        city4_geo, city5_geo, city6_geo,
        city7_geo, city8_geo, city9_geo, city10_geo]

In [250]:
climate = [city1_climate, city2_climate, city3_climate, 
           city4_climate, city5_climate, city6_climate,
           city7_climate, city8_climate, city9_climate, city10_climate]

In [251]:
top_city_table['City Summary'] = summary

In [252]:
top_city_table['City Geography'] = geo

In [253]:
top_city_table['City Climate'] = climate

,City Name,State,Wiki Link,Latitude,Longitude,City Size,City Population,City Summary,City Geography,City Climate
0,New York City,New York,https://en.wikipedia.org/wiki/New_York_City,40°39′49″N,40°39′49″N,301.5 sq mi,"8,398,748","[<p>The <b>City of New York</b>, usually calle...",[<p>New York City is situated in the <a class=...,"[<p>Under the <a href=""/wiki/K%C3%B6ppen_clima..."
1,Los Angeles,California,https://en.wikipedia.org/wiki/Los_Angeles,34°01′10″N,34°01′10″N,468.7 sq mi,"3,990,456","[<p><b>Los Angeles</b> (<span class=""nowrap""><...",[<p>Los Angeles is both flat and hilly. The hi...,"[<p>Los Angeles has a <a href=""/wiki/Mediterra..."
2,Chicago,Illinois,https://en.wikipedia.org/wiki/Chicago,41°50′15″N,41°50′15″N,227.3 sq mi,"2,705,994","[<p><b>Chicago</b> (<span class=""nowrap""><span...",[<p>Chicago is located in northeastern Illinoi...,[<p>The city lies within the hot-summer <a hre...
3,Houston,Texas,https://en.wikipedia.org/wiki/Houston,29°47′12″N,29°47′12″N,637.5 sq mi,"2,325,502",[<p>Comprising a total area of 637.4 square mi...,"[<p>Houston has four major <a class=""mw-redire...",[<p>Prevailing winds are from the south and so...
4,Phoenix,Arizona,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",33°34′20″N,33°34′20″N,517.6 sq mi,"1,445,632","[<p class=""mw-empty-elt""> </p>, <p><b>Phoenix<...",[<p>Phoenix is in the southwestern United Stat...,"[<p>Phoenix has a <a href=""/wiki/Desert_climat..."
5,Philadelphia,Pennsylvania,https://en.wikipedia.org/wiki/Philadelphia,40°00′34″N,40°00′34″N,134.2 sq mi,"1,584,138","[<p><b>Philadelphia</b> (<a class=""mw-redirect...",[<p>The geographic center of Philadelphia is l...,"[<p>According to the <a href=""/wiki/K%C3%B6ppe..."
6,San Antonio,Texas,https://en.wikipedia.org/wiki/San_Antonio,29°28′21″N,29°28′21″N,461.0 sq mi,"1,532,233","[<p><b>San Antonio</b> (<span class=""nowrap""><...",[<p>The San Antonio Missions National Historic...,"[<p>San Antonio has a transitional <a href=""/w..."
7,San Diego,California,https://en.wikipedia.org/wiki/San_Diego,32°48′55″N,32°48′55″N,325.2 sq mi,"1,425,976","[<p><b>San Diego</b> (<span class=""nowrap""><sp...",[<p>According to SDSU professor emeritus Monte...,[<p>San Diego has one of the top-ten best clim...
8,Dallas,Texas,https://en.wikipedia.org/wiki/Dallas,32°47′36″N,32°47′36″N,340.9 sq mi,"1,345,047","[<p><b>Dallas</b> (<span class=""nowrap""><span ...","[<p>Dallas is situated in the <a href=""/wiki/S...","[<p>Dallas has a <a href=""/wiki/Humid_subtropi..."
9,San Jose,California,"https://en.wikipedia.org/wiki/San_Jose,_Califo...",37°17′48″N,37°17′48″N,177.5 sq mi,"1,030,119","[<p><b>San Jose</b><sup class=""reference"" id=""...","[<p>San Jose is located at <span class=""plainl...","[<p>San Jose, like most of the Bay Area, has a..."


In [255]:
top_city_table.to_csv('dataset.csv', index=False)

## 6. BigQueryTable

In [261]:
from google.cloud import bigquery
client = bigquery.Client()
filename = './dataset.csv'

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(
        source_file,
        location='US',
        job_config=job_config,
    )

job.result()


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started